In [46]:
import os
os.chdir("/diazlab/data3/.abhinav/projects/SHH/Visium/3rd_strict_cutoff/python/")
import importlib
import preprocessing
importlib.reload(preprocessing)

<module 'preprocessing' from '/diazlab/data3/.abhinav/projects/SHH/Visium/3rd_strict_cutoff/python/preprocessing.py'>

In [48]:
# We would also like to have dataframe before and after filtering
import pandas as pd

results_list = []

file_names = [
    "SHH_SF03521.h5ad", "SHH_SF07994.h5ad", "SHH_SF08539.h5ad",
    "SHH_SF09782.h5ad", "SHH_SF10067.h5ad", "SHH_SF10210.h5ad", 
    "SHH_SF12434.h5ad"
    ]

# Input files and output directory paths
input_dir = "/diazlab/data3/.abhinav/projects/SHH/Visium/scanpy/"
output_dir = "/diazlab/data3/.abhinav/projects/SHH/Visium/3rd_strict_cutoff/python/"

# Loop over each sample 
for file_name in file_names:
    input_file = input_dir + file_name
    output_prefix = file_name.replace(".h5ad", "")  # Remove file extension to use as output prefix

    processed_data,cells_before,cells_after,genes_before,genes_after = preprocessing.process_and_visualize_data(input_file, output_dir, output_prefix)

    results_list.append({
        "Sample": output_prefix,
        "cells_before": cells_before,
        "cells_after": cells_after,
        "genes_before": genes_before,
        "genes_after": genes_after
    })

    
    # Store the processed data
    globals()[output_prefix] = processed_data

# Create a DataFrame from the results list
results_df = pd.DataFrame(results_list)

# Save the results to a CSV file
os.makedirs(os.path.join(output_dir,"Table"), exist_ok=True)
results_df.to_csv(os.path.join(output_dir, "Table/cell_counts_summary.csv"), index=False)

Cells after min genes > 500 and MT filter < 20 filter: 1563
Genes after min cells filter = 10: 11265
Cells after min genes > 500 and MT filter < 20 filter: 950
Genes after min cells filter = 10: 10651
Cells after min genes > 500 and MT filter < 20 filter: 580
Genes after min cells filter = 10: 10488
Cells after min genes > 500 and MT filter < 20 filter: 1819
Genes after min cells filter = 10: 13047
Cells after min genes > 500 and MT filter < 20 filter: 2253
Genes after min cells filter = 10: 11778
Cells after min genes > 500 and MT filter < 20 filter: 2853
Genes after min cells filter = 10: 13120
Cells after min genes > 500 and MT filter < 20 filter: 2225
Genes after min cells filter = 10: 14503


In [49]:
file_names = ["SF08539_2","SF09782_2"]

# Define the paths for input files and output directory
input_dirs = ["/diazlab/data3/.abhinav/projects/SHH/Visium/2nd/SF08539/outs/", 
             "/diazlab/data3/.abhinav/projects/SHH/Visium/2nd/SF09782/outs/"]

output_dir = "/diazlab/data3/.abhinav/projects/SHH/Visium/3rd_strict_cutoff/python/"

for input_dir, file_name in zip(input_dirs, file_names):
    processed_data,cells_before,cells_after,genes_before,genes_after = preprocessing.process_and_visualize_data(input_dir, output_dir, file_name)
    globals()[file_name] = processed_data

    results_list.append({
        "Sample": file_name,
        "cells_before": cells_before,
        "cells_after": cells_after,
        "genes_before": genes_before,
        "genes_after": genes_after
    })

    
    # Store the processed data
    globals()[output_prefix] = processed_data

# Create a DataFrame from the results list
results_df = pd.DataFrame(results_list)

# Save the results to a CSV file
os.makedirs(os.path.join(output_dir,"Table"), exist_ok=True)
results_df.to_csv(os.path.join(output_dir, "Table/cell_counts_summary.csv"), index=False)


Cells after min genes > 500 and MT filter < 20 filter: 797
Genes after min cells filter = 10: 9988
Cells after min genes > 500 and MT filter < 20 filter: 2871
Genes after min cells filter = 10: 13029


### Performing all the sample integration

Removing the low quality samples

In [1]:
## Loading Libraries
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [51]:
# Filter out internal Python variables
user_vars = [var for var in dir() if not var.startswith('_')]
sf_vars = [var for var in user_vars if 'SF' in var]
print(sf_vars)

['SF08539_2', 'SF09782_2', 'SHH_SF03521', 'SHH_SF07994', 'SHH_SF08539', 'SHH_SF09782', 'SHH_SF10067', 'SHH_SF10210', 'SHH_SF12434']


In [53]:
# Correcting for the batch effect
import scanorama
adatas = [SF09782_2, SHH_SF03521, SHH_SF07994, SHH_SF08539, SHH_SF09782, SHH_SF10067, SHH_SF10210, SHH_SF12434]
adata_cor = scanorama.correct_scanpy(adatas, return_dimred=True)

Found 8441 genes among all datasets
[[0.         0.         0.         0.00862069 0.3793293  0.
  0.         1.        ]
 [0.         0.         0.25052632 0.01206897 0.         0.02885042
  0.621881   0.        ]
 [0.         0.         0.         0.03793103 0.01473684 0.0497115
  0.02315789 0.        ]
 [0.         0.         0.         0.         0.16206897 0.20689655
  0.         0.00344828]
 [0.         0.         0.         0.         0.         0.05104305
  0.         0.34964266]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]]
Processing datasets (0, 7)
Processing datasets (1, 6)
Processing datasets (0, 4)
Processing datasets (4, 7)
Processing datasets (1, 2)
Processing datasets (3, 5)
Processing datasets (3, 4)


In [131]:
SF09782_2

AnnData object with n_obs × n_vars = 2871 × 13029
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_genes', 'clusters'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'spatial', 'log1p', 'hvg', 'pca', 'neighbors', 'umap', 'clusters', 'clusters_colors', 'rank_genes_groups', 'dendrogram_clusters'
    obsm: 'spatial', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [81]:
    keys=[
        k
        for d in [
            adata_cor[0].uns["spatial"],
            adata_cor[1].uns["spatial"],
            adata_cor[2].uns["spatial"],
            adata_cor[3].uns["spatial"],
            adata_cor[4].uns["spatial"],
            adata_cor[5].uns["spatial"],
            adata_cor[6].uns["spatial"],
            adata_cor[7].uns["spatial"]
        ]
        for k, v in d.items()
    ]

In [125]:
# Extract keys from each adata_cor's uns["spatial"] and store them in a list
all_keys = []
for i in range(8):  # Iterate from 0 to 7
    spatial_keys = adata_cor[i].uns["spatial"].keys()  # Get the keys of the spatial dictionary
    for key in spatial_keys:
        all_keys.append(f"{key}_{i}")  # Append index to make the keys unique
all_keys


['SF09782_0',
 'SF03521_1',
 'SF07994_2',
 'SF08539_3',
 'SF09782_4',
 'SF10067_5',
 'SF10210_6',
 'SF09782_7']

In [ ]:
## We will concatenate the two datasets and save the integrated embeddings in adata_spatial.obsm['scanorama_embedding']. 
# Furthermore we will compute UMAP to visualize the results and qualitatively assess the data integration task.

# Notice that we are concatenating the dataset with uns_merge="unique" strategy, in order to keep both images 
# from the visium datasets in the concatenated anndata object.

adata_spatial = sc.concat(
    adata_cor,
    label="library_id",
    uns_merge="unique",
    keys=all_keys,
    index_unique="-",
)

sc.pp.neighbors(adata_spatial, use_rep="X_scanorama")
sc.tl.umap(adata_spatial)
sc.tl.leiden(adata_spatial, key_added="clusters")

In [134]:
sc.pl.umap(adata_spatial, 
           color=["clusters", "library_id"], 
           size=30, 
           save="_combined.pdf", 
           show = False
)

[<Axes: title={'center': 'clusters'}, xlabel='UMAP1', ylabel='UMAP2'>,
 <Axes: title={'center': 'library_id'}, xlabel='UMAP1', ylabel='UMAP2'>]

In [ ]:

# We can also visualize the clustering result in spatial coordinates. For that, we first need to save the cluster colors in a 
# dictionary. We can then plot the Visium tissue fo the Anterior and Posterior Sagittal view, alongside each other
clusters_colors = dict(
    zip([str(i) for i in range(18)], adata_spatial.uns["clusters_colors"])
)

fig, axs = plt.subplots(1, 4, figsize=(20, 5))

for i, library in enumerate(
    ['slide_018_A1_R', 'slide_018_D11_R', 'slide_075_A1', 'slide_075_D1']
):
    ad = adata_spatial[adata_spatial.obs.library_id == library, :].copy()
    sc.pl.spatial(
        ad,
        img_key="hires",
        library_id=library,
        color="clusters",
        bw=False,
        alpha_img=0,
        size=1.3,
        palette=[
            v
            for k, v in clusters_colors.items()
            if k in ad.obs.clusters.unique().tolist()
        ],
        legend_loc=None,
        show=False,
        ax=axs[i],
    )

plt.savefig("spatial_clusters.png")

sc.pp.normalize_total(adata_spatial, inplace=True)
sc.pp.log1p(adata_spatial)

### Find the Markers in these clusters
### Cluster marker genes
sc.tl.rank_genes_groups(adata_spatial, "clusters", method="t-test")
sc.pl.rank_genes_groups_heatmap(adata_spatial, n_genes=5, groupby="clusters", show_gene_labels=True)
plt.savefig("heatmap_markers.png")

import pandas as pd

# Assuming adata_spatial.uns['rank_genes_groups'] is your dictionary
rank_genes_groups_dict = adata_spatial.uns['rank_genes_groups']

# Extract relevant arrays from the dictionary
names_array = rank_genes_groups_dict['names']
scores_array = rank_genes_groups_dict['scores']
pvals_array = rank_genes_groups_dict['pvals']
pvals_adj_array = rank_genes_groups_dict['pvals_adj']
logfoldchanges_array = rank_genes_groups_dict['logfoldchanges']

pd.DataFrame((names_array)) ### this show each clusters has different gene names as told to bottom markers and so on for scores, pvalue, etc. 
genes = ["MGP","COL1A2","IGHG3","IGHA1","COL3A1"]

plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata_spatial, img_key="hires", color=["MT-CO2","MGP","COL1A2","IGHG3","IGHA1","COL3A1","CLEC4E","SIRPA"], bw=False, alpha_img=0, size=1.3)
plt.savefig(savedir + "spatial_topgenes.png")
